# Fetch Live Data

In [138]:
import requests

In [139]:
def fetch_crypto_data():
    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {
        "vs_currency": "usd",          # Get prices in USD
        "order": "market_cap_desc",    # Order by market capitalization
        "per_page": 50,                # Number of cryptocurrencies to fetch
        "page": 1,                     # Fetch the first page of results
        "sparkline": False             # Exclude sparkline data
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Error: Unable to fetch data (Status Code: {response.status_code})")
        return None

In [140]:
crypto_data = fetch_crypto_data()
if crypto_data:
    print("Top 50 Cryptocurrencies:")
    for i, coin in enumerate(crypto_data, start=1):
        print(f"{i}. {coin['name']} ({coin['symbol'].upper()}) - ${coin['current_price']} {coin['market_cap']} {coin['total_volume']} {coin['price_change_percentage_24h']}")

Top 50 Cryptocurrencies:
1. Bitcoin (BTC) - $92554 1842464186193 61624688461 -1.4207
2. Ethereum (ETH) - $3242.31 392321181635 28874942687 -0.26736
3. Tether (USDT) - $0.999876 137583768271 87594299741 -0.04711
4. XRP (XRP) - $2.3 132572294612 7220979063 1.49854
5. BNB (BNB) - $686.1 100446366337 1111024322 -0.24098
6. Solana (SOL) - $187.38 90760262424 4543727118 -2.55681
7. Dogecoin (DOGE) - $0.320746 47483155150 4125696956 -3.96223
8. USDC (USDC) - $1.0 45362539038 9855510746 -0.11609
9. Cardano (ADA) - $0.911398 32874831163 1693746187 -3.19907
10. Lido Staked Ether (STETH) - $3241.11 31203024530 62540308 -0.28697
11. TRON (TRX) - $0.241554 20873997211 1255613984 -2.20706
12. Avalanche (AVAX) - $36.44 14984034042 441812824 -1.94722
13. Sui (SUI) - $4.74 14330828358 2093132260 3.90427
14. Wrapped stETH (WSTETH) - $3894.49 13740238051 58525584 0.20122
15. Toncoin (TON) - $5.17 13129172734 255792707 -1.27181
16. Shiba Inu (SHIB) - $2.139e-05 12587251639 569884191 1.92526
17. Chainlink 

# Data Analysis

In [141]:
def analyze_crypto_data(data):
    
    # Identify the top 5 cryptocurrencies by market capitalization
    top_5_by_market_cap = sorted(data, key=lambda x: x['market_cap'], reverse=True)[:5]

    # Calculate the average price of the top 50 cryptocurrencies
    average_price = sum(coin['current_price'] for coin in data) / len(data)

    # Find the highest and lowest percentage change in 24 hours
    highest_change = max(data, key=lambda x: x['price_change_percentage_24h'])
    lowest_change = min(data, key=lambda x: x['price_change_percentage_24h'])

    # Display the analysis results
    print("\n--- Data Analysis ---")
    print("\nTop 5 Cryptocurrencies by Market Cap:")
    i=1
    for coin in top_5_by_market_cap:
        print(f"{i}. {coin['name']} ({coin['symbol'].upper()}) - Market Cap: ${coin['market_cap']:,}")
        i += 1

    print(f"\nAverage Price of Top 50 Cryptocurrencies: ${average_price:.2f}")
    print(f"\nHighest 24h % Change: {highest_change['name']} ({highest_change['price_change_percentage_24h']:.2f}%)")
    print(f"Lowest 24h % Change: {lowest_change['name']} ({lowest_change['price_change_percentage_24h']:.2f}%)")

In [142]:
analyze_crypto_data(crypto_data)


--- Data Analysis ---

Top 5 Cryptocurrencies by Market Cap:
1. Bitcoin (BTC) - Market Cap: $1,842,464,186,193
2. Ethereum (ETH) - Market Cap: $392,321,181,635
3. Tether (USDT) - Market Cap: $137,583,768,271
4. XRP (XRP) - Market Cap: $132,572,294,612
5. BNB (BNB) - Market Cap: $100,446,366,337

Average Price of Top 50 Cryptocurrencies: $4095.81

Highest 24h % Change: Sui (3.90%)
Lowest 24h % Change: Virtuals Protocol (-7.82%)


# Live-Running Excel Sheet

In [143]:
import openpyxl
import time
from openpyxl.styles import PatternFill
import os
from fpdf import FPDF

In [144]:
def update_excel_sheet(data):
    file_path = 'crypto_data.xlsx'
    
    if os.path.exists(file_path):
        workbook = openpyxl.load_workbook(file_path)
        sheet = workbook.active
    else:
        workbook = openpyxl.Workbook()
        sheet = workbook.active
        sheet['A1'] = 'Cryptocurrency Name'
        sheet['B1'] = 'Symbol'
        sheet['C1'] = 'Current Price (USD)'
        sheet['D1'] = 'Market Capitalization'
        sheet['E1'] = '24-hour Trading Volume'
        sheet['F1'] = 'Price Change (24h, %)'

    # Create fill styles for row highlighting
    red_fill = PatternFill(start_color='FFCCCB', end_color='FFCCCB', fill_type='solid')  # Red for decrease
    green_fill = PatternFill(start_color='90EE90', end_color='90EE90', fill_type='solid')  # Green for increase

    for coin in data:
        # Check if the coin already exists in the sheet by its name (column A)
        existing_row = None
        for row in range(2, sheet.max_row + 1):  # Skip header row
            if sheet[f'A{row}'].value == coin['name']:
                existing_row = row
                break

        if existing_row:
            # Update existing row
            sheet[f'C{existing_row}'] = coin['current_price']
            sheet[f'D{existing_row}'] = coin['market_cap']
            sheet[f'E{existing_row}'] = coin['total_volume']
            sheet[f'F{existing_row}'] = coin['price_change_percentage_24h']

            price_change = coin['price_change_percentage_24h']
            
            # Apply color fill based on price change
            fill = green_fill if price_change >= 0 else red_fill
            for col in range(1, 7):  # Columns A to F
                sheet.cell(row=existing_row, column=col).fill = fill
        else:
            # Add new row for the cryptocurrency
            row = sheet.max_row + 1
            sheet[f'A{row}'] = coin['name']
            sheet[f'B{row}'] = coin['symbol'].upper()
            sheet[f'C{row}'] = coin['current_price']
            sheet[f'D{row}'] = coin['market_cap']
            sheet[f'E{row}'] = coin['total_volume']
            sheet[f'F{row}'] = coin['price_change_percentage_24h']

            price_change = coin['price_change_percentage_24h']
            
            # Apply color fill based on price change
            fill = green_fill if price_change >= 0 else red_fill
            for col in range(1, 7):  # Columns A to F
                sheet.cell(row=row, column=col).fill = fill

    workbook.save(file_path)

In [145]:
def generate_analysis_report(data):
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()

    pdf.set_font('Arial', 'B', 16)
    pdf.cell(200, 10, txt="Cryptocurrency Market Analysis", ln=True, align='C')

    pdf.set_font('Arial', '', 12)
    pdf.ln(10)
    pdf.cell(200, 10, txt="Key Insights and Analysis", ln=True)

    pdf.ln(10)
    pdf.set_font('Arial', '', 10)
    pdf.cell(30, 10, 'Name', border=1)
    pdf.cell(30, 10, 'Symbol', border=1)
    pdf.cell(50, 10, 'Current Price (USD)', border=1)
    pdf.cell(50, 10, '24h Change (%)', border=1)
    pdf.ln(10)

    for coin in data:
        pdf.cell(30, 10, coin['name'], border=1)
        pdf.cell(30, 10, coin['symbol'], border=1)
        pdf.cell(50, 10, f"${coin['current_price']}", border=1)
        pdf.cell(50, 10, f"{coin['price_change_percentage_24h']}%", border=1)
        pdf.ln(10)

    pdf.output("analysis_report.pdf")

In [146]:
def run_live_updates():
    while True:
        crypto_data = fetch_crypto_data()
        if crypto_data:
            update_excel_sheet(crypto_data)
            #generate_analysis_report(crypto_data)
            print("Updated")
        time.sleep(60)  # Wait for 5 minutes (300 seconds)

In [ ]:
run_live_updates()

Updated
Updated
Updated
Updated
Updated
Updated
Updated
Updated
Updated
Updated
Updated
Updated
Updated
Updated
Updated
Updated
Updated
Updated
Updated
Updated
Updated
Updated
